In [1]:
from sage.all import sigma, nth_prime
from sage.arith.all import moebius
import pickle
import ast

print(nth_prime(400))

2741


In [3]:
def get_all_newforms_at_level(level, expansion_terms=30, output_file=None):
    """
    Get ALL weight 2 rational newforms at a specific level.
    Only takes one curve per isogeny class (since they give the same newform).
    """
    from sage.databases.cremona import CremonaDatabase
    import time
    
    start_time = time.time()
    db = CremonaDatabase()
    
    rational_forms = []
    
    print(f"Weight 2 rational newforms at level {level}")
    print(f"Computing {expansion_terms} terms\n")
    
    # Open output file if specified
    if output_file:
        f_out = open(output_file, 'w')
    
    try:
        curves = db.allcurves(level)
        
        if curves:
            # Group by isogeny class (the letter part)
            isogeny_classes = {}
            for label in curves.keys():
                # Extract the letter (isogeny class identifier)
                letter = ''.join([c for c in label if c.isalpha()])
                if letter not in isogeny_classes:
                    isogeny_classes[letter] = label
            
            print(f"Found {len(isogeny_classes)} distinct newforms (isogeny classes)\n")
            
            # Process one curve per isogeny class
            for letter in sorted(isogeny_classes.keys()):
                label = isogeny_classes[letter]
                full_label = f"{level}{label}"
                E = EllipticCurve(full_label)
                rational_forms.append((level, 2, E))
                
                # Print header to console
                print(f"\n{'='*70}")
                print(f"Newform {len(rational_forms)}: Isogeny class {level}{letter}")
                print(f"Representative curve: {full_label}")
                print(f"{'='*70}")
                
                # Get coefficients
                an_list = E.anlist(expansion_terms + 1)
                coeffs = an_list[1:]
                
                # Write to file
                if output_file:
                    f_out.write(f"Isogeny class: {level}{letter}\n")
                    f_out.write(f"Representative curve: {full_label}\n")
                    f_out.write(f"Coefficients a_1 to a_{expansion_terms}:\n")
                    f_out.write(str(coeffs) + "\n\n")
                
                # Show first 10 coefficients to console
                print(f"Coefficients (first 10 of {expansion_terms}):")
                print(f"  a_1-a_10: {coeffs[:10]}")
                
                # Additional info
                print(f"\nCurve info:")
                print(f"  Conductor: {E.conductor()}")
                print(f"  Rank: {E.rank()}")
                
                if output_file:
                    f_out.write(f"Conductor: {E.conductor()}\n")
                    f_out.write(f"Rank: {E.rank()}\n")
                    f_out.write(f"{'='*70}\n\n")
    
    except Exception as e:
        print(f"Error at level {level}: {e}")
    
    if output_file:
        f_out.close()
        print(f"\nFull data written to: {output_file}")
    
    elapsed = time.time() - start_time
    print(f"\n{'='*70}")
    print(f"Found {len(rational_forms)} distinct newforms in {elapsed:.2f} seconds")
    print(f"{'='*70}")
    
    return rational_forms


results = get_all_newforms_at_level(
    level=27,
    expansion_terms=2800,
    output_file="/Users/barrybrent/data2/run1mar26no8.txt"
)

Weight 2 rational newforms at level 27
Computing 2800 terms

Found 1 distinct newforms (isogeny classes)


Newform 1: Isogeny class 27a
Representative curve: 27a1
Coefficients (first 10 of 2800):
  a_1-a_10: [1, 0, 0, -2, 0, 0, -1, 0, 0, 0]

Curve info:
  Conductor: 27
  Rank: 0

Full data written to: /Users/barrybrent/data2/run1mar26no8.txt

Found 1 distinct newforms in 0.01 seconds


In [4]:
import pickle
# loads all forms at a given level one at a time converted to lists
def load_newforms_from_file(filename):
    """
    Load all newforms from a file and return them as a list of dictionaries.
    Each dictionary contains the label and coefficients.
    """
    with open(filename, 'r') as f:
        content = f.read()
    
    # Split by the separator line
    blocks = content.split('='*70)
    
    newforms = []
    
    for block in blocks:
        if 'Isogeny class:' in block:
            lines = block.strip().split('\n')
            
            # Extract isogeny class
            isogeny_class = lines[0].replace('Isogeny class:', '').strip()
            
            # Extract representative curve
            rep_curve = lines[1].replace('Representative curve:', '').strip()
            
            # Find the coefficients line
            for i, line in enumerate(lines):
                if line.startswith('['):
                    coeffs = eval(line)
                    break
            
            newforms.append({
                'isogeny_class': isogeny_class,
                'representative': rep_curve,
                'coefficients': coeffs
            })
    
    return newforms

# Load all newforms
newforms = load_newforms_from_file("/Users/barrybrent/data2/run1mar26no8.txt")
print("curve 27a1")
print(f"Loaded {len(newforms)} newforms\n")
print("number of newforms:",len(newforms))
# Access them one at a time
for i, form in enumerate(newforms):
    print(f"Newform {i+1}:")
    print(f"  Isogeny class: {form['isogeny_class']}")
    print(f"  Representative: {form['representative']}")
    print(f"  First 10 coefficients: {form['coefficients'][:10]}")
    print(f"  Total coefficients: {len(form['coefficients'])}")
    print()

# Access individual newforms
newform_1 = newforms[0]['coefficients']  # First newform
#newform_2 = newforms[1]['coefficients']  # Second newform (if it exists)

print(f"Newform 1, coefficient of q^5: {newform_1[4]}")  # Remember: index 4 is a_5
print(newform_1[:10])

def curve_twentyseven_prime(n):
    return newform_1[nth_prime(n)]

curve_twentyseven_prime_list=[curve_twentyseven_prime(n) for n in range(1,401)]
with open('/Users/barrybrent/data2/run1mar26no9.txt', 'wb') as wfile:
    pickle.dump(curve_twentyseven_prime_list, wfile) 


curve 27a1
Loaded 1 newforms

number of newforms: 1
Newform 1:
  Isogeny class: 27a
  Representative: 27a1
  First 10 coefficients: [1, 0, 0, -2, 0, 0, -1, 0, 0, 0]
  Total coefficients: 2801

Newform 1, coefficient of q^5: 0
[1, 0, 0, -2, 0, 0, -1, 0, 0, 0]


In [5]:
from sage.all import sigma, nth_prime
from sage.arith.all import moebius
import pickle
import ast

with open('/Users/barrybrent/data2/run1mar26no9.txt', 'rb') as rfile:
    curve_twentyseven_prime_list = pickle.load(rfile)



h_list=[1]+curve_twentyseven_prime_list # h_0 = 1 required by the lemmas
j_list=[(0,1)] # first entry is just an offset. 
for n in range(1,401):# so j(1) = 1 = h(0) as required.
    sm=0
    for r in [1..n-1]: 
        sm=sm+j_list[r][1]*h_list[n-r] 
    j_n=n*h_list[n]-sm
    print((n,j_n))
    j_list.append((n,j_n))
    
with open('/Users/barrybrent/data2/run1mar26no10.txt', 'wb') as wfile:
    pickle.dump(j_list, wfile) ##<<<<<<<<<<<<<<<<alternating zeros & - 2^n

(1, 0)
(2, -4)
(3, 0)
(4, -8)
(5, 0)
(6, -16)
(7, 0)
(8, -32)
(9, 0)
(10, -64)
(11, 0)
(12, -128)
(13, 0)
(14, -256)
(15, 0)
(16, -512)
(17, 0)
(18, -1024)
(19, 0)
(20, -2048)
(21, 0)
(22, -4096)
(23, 0)
(24, -8192)
(25, 0)
(26, -16384)
(27, 0)
(28, -32768)
(29, 0)
(30, -65536)
(31, 0)
(32, -131072)
(33, 0)
(34, -262144)
(35, 0)
(36, -524288)
(37, 0)
(38, -1048576)
(39, 0)
(40, -2097152)
(41, 0)
(42, -4194304)
(43, 0)
(44, -8388608)
(45, 0)
(46, -16777216)
(47, 0)
(48, -33554432)
(49, 0)
(50, -67108864)
(51, 0)
(52, -134217728)
(53, 0)
(54, -268435456)
(55, 0)
(56, -536870912)
(57, 0)
(58, -1073741824)
(59, 0)
(60, -2147483648)
(61, 0)
(62, -4294967296)
(63, 0)
(64, -8589934592)
(65, 0)
(66, -17179869184)
(67, 0)
(68, -34359738368)
(69, 0)
(70, -68719476736)
(71, 0)
(72, -137438953472)
(73, 0)
(74, -274877906944)
(75, 0)
(76, -549755813888)
(77, 0)
(78, -1099511627776)
(79, 0)
(80, -2199023255552)
(81, 0)
(82, -4398046511104)
(83, 0)
(84, -8796093022208)
(85, 0)
(86, -17592186044416)
(

In [27]:
def rdr(p,n):
    return QQ.valuation(p)(n)
no=[]
import pickle
with open('/Users/barrybrent/data2/run1mar26no10.txt', 'rb') as rfile:
    j_list = pickle.load(rfile)
print("length: ",len(j_list))
for pair in j_list:
    n = pair[0]
    jj=pair[1]
    if n%2==1:
        if jj!=0:
            no.append(n)
    if n>0:
        if n%2==0:
            if jj!=-2*2**(n/2):
                no.append(n)
print(len(no))
                
            
    

length:  401
0
